# LOOCV Error Estimate

In [28]:
from ISLP import load_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Data

In [29]:
data = load_data('Weekly')

data

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up
...,...,...,...,...,...,...,...,...,...
1084,2010,-0.861,0.043,-2.173,3.599,0.015,3.205160,2.969,Up
1085,2010,2.969,-0.861,0.043,-2.173,3.599,4.242568,1.281,Up
1086,2010,1.281,2.969,-0.861,0.043,-2.173,4.835082,0.283,Up
1087,2010,0.283,1.281,2.969,-0.861,0.043,4.454044,1.034,Up


In [33]:
x = data[['Lag1', 'Lag2']].values
y = np.array([1 if d=="Up" else 0 for d in data['Direction']])

[[ 0.816  1.572]
 [-0.27   0.816]
 [-2.576 -0.27 ]
 ...
 [ 1.281  2.969]
 [ 0.283  1.281]
 [ 1.034  0.283]]


## LOOCV Loop


In [34]:
def LOOCV_acc_score(x,y, idx):
    x_test,y_test = x[idx], y[idx]

    mask = np.ones(len(x), dtype=bool)
    mask[idx] = False
    x_train, y_train = x[mask],y[mask]

    model = LogisticRegression()
    model.fit(x_train, y_train)

    preds = model.predict(x_test)

    return accuracy_score(y_test,preds)


In [35]:
acc_scores = []
for i in range(len(y)):
    acc_scores.append(LOOCV_acc_score(x,y,i))

plt.hist(acc_scores)

[[-0.27   0.816]
 [-2.576 -0.27 ]
 [ 3.514 -2.576]
 ...
 [ 1.281  2.969]
 [ 0.283  1.281]
 [ 1.034  0.283]]


ValueError: Expected 2D array, got 1D array instead:
array=[0.816 1.572].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.